<a href="https://colab.research.google.com/github/bmnds/uea-data-science-04-data-processing/blob/main/Atividade02_Transformando_dados_com_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# JUNÇÃO E CLASSIFICAÇÃO
*   Ler os dados de diferentes tabelas;
*   Extrair variável de interesse;
*   Aplicar transformações.



In [ ]:
dt_pop = pd.read_csv('/content/drive/MyDrive/UEA_POS_CD/modulo4/trabalho1/brazil_population_2019.csv',error_bad_lines=False)
dt_pop.head() 

b'Skipping line 1282: expected 8 fields, saw 9\nSkipping line 1283: expected 8 fields, saw 9\nSkipping line 1293: expected 8 fields, saw 9\nSkipping line 1294: expected 8 fields, saw 9\nSkipping line 1300: expected 8 fields, saw 9\nSkipping line 1308: expected 8 fields, saw 9\nSkipping line 1309: expected 8 fields, saw 9\nSkipping line 1314: expected 8 fields, saw 9\nSkipping line 1319: expected 8 fields, saw 9\nSkipping line 1320: expected 8 fields, saw 9\nSkipping line 1321: expected 8 fields, saw 9\nSkipping line 1323: expected 8 fields, saw 9\n'


,region,state,city,state_code,city_code,health_region_code,health_region,population
0,Centro-Oeste,Mato Grosso do Sul,Alcinópolis,50,500025,50001,Campo Grande,5343
1,Centro-Oeste,Mato Grosso do Sul,Amambai,50,500060,50003,Dourados,39396
2,Centro-Oeste,Mato Grosso do Sul,Bataguassu,50,500190,50004,Três Lagoas,23024
3,Centro-Oeste,Mato Grosso do Sul,Batayporã,50,500200,50003,Dourados,11329
4,Centro-Oeste,Mato Grosso do Sul,Bela Vista,50,500210,50001,Campo Grande,24629


In [ ]:
regions = np.unique(dt_pop['region'])
regions

array(['Centro-Oeste', 'Nordeste', 'Norte', 'Sudeste', 'Sul'],
      dtype=object)

In [ ]:
all_regions_pop = [ dt_pop['population'].loc[dt_pop["region"]== region].sum() for region in regions ]
all_regions_pop

regions_pop = [('Centro-Oeste',all_regions_pop[0]),('Nordeste',all_regions_pop[1]),\
                      ('Norte',all_regions_pop[2]),('Sudeste',all_regions_pop[3]),\
                      ('Sul',all_regions_pop[4])]

regions_population = pd.DataFrame.from_dict(dict(regions_pop),orient='index',columns=['population'])
regions_population

,population
Centro-Oeste,13085364
Nordeste,48946190
Norte,14752784
Sudeste,80421777
Sul,24522343


In [ ]:
dt_cases = pd.read_csv('/content/drive/MyDrive/UEA_POS_CD/modulo4/trabalho1/brazil_covid19.csv',error_bad_lines=False)
dt_cases.head() 

,date,region,state,cases,deaths
0,2020-02-25,Centro-Oeste,DF,0,0
1,2020-02-25,Centro-Oeste,GO,0,0
2,2020-02-25,Centro-Oeste,MS,0,0
3,2020-02-25,Centro-Oeste,MT,0,0
4,2020-02-25,Nordeste,AL,0,0


In [ ]:
"""
Encontrando total de óbitos por região
"""
all_regions_deaths = [ dt_cases['deaths'].loc[dt_cases["region"]== region].sum() for region in regions ]
all_regions_deaths

regions_deaths = [('Centro-Oeste',all_regions_deaths[0]),('Nordeste',all_regions_deaths[1]),\
                      ('Norte',all_regions_deaths[2]),('Sudeste',all_regions_deaths[3]),\
                      ('Sul',all_regions_deaths[4])]

regions_deaths = pd.DataFrame.from_dict(dict(regions_deaths),orient='index',columns=['deaths'])
regions_deaths

,deaths
Centro-Oeste,1518353
Nordeste,5812513
Norte,2386633
Sudeste,9326839
Sul,1464191


In [ ]:
"""
Encontrando total de casos por região
"""

all_regions_cases = [ dt_cases['cases'].loc[dt_cases["region"]== region].sum() for region in regions ]

regions_cases = [('Centro-Oeste',all_regions_cases[0]),('Nordeste',all_regions_cases[1]),\
                      ('Norte',all_regions_cases[2]),('Sudeste',all_regions_cases[3]),\
                      ('Sul',all_regions_cases[4])]

regions_cases = pd.DataFrame.from_dict(dict(regions_cases),orient='index',columns=['cases'])
regions_cases

,cases
Centro-Oeste,71590231
Nordeste,184449738
Norte,89017773
Sudeste,224057624
Sul,72207079


## JUNÇÃO

> Realiza-se a junção do arquivo *brazil_population_2019.csv*, contendo a informação populacional com *brazil_covid19.csv*, a qual informa o número de casos e óbitos, a fim de posteriormente aplicar classificação.

In [ ]:
dt = pd.concat([regions_population,regions_cases],axis=1)
dt = pd.concat([dt,regions_deaths],axis=1)
dt

,population,cases,deaths
Centro-Oeste,13085364,71590231,1518353
Nordeste,48946190,184449738,5812513
Norte,14752784,89017773,2386633
Sudeste,80421777,224057624,9326839
Sul,24522343,72207079,1464191


## CLASSIFICAÇÃO

* Classificar porcentagem de óbitos por região.
* Classificar porcentagem de casos de cura.
* > Obs.: A data compreende o periodo de 2020-02-25 a 2020-11-25


* A coluna ***deaths_percent*** representa a porcentagem de óbitos da população.
* A coluna *safe_cases* representa a porcentagem de casos detectados e curados.

In [ ]:
"""
Ordenando por porcentagem de óbitos
"""

dt['deaths_percent'] = (dt['deaths']/dt['population'])*100
dt['safe_cases'] = ((dt['cases']-dt['deaths'])/dt['cases'])*100
dt.sort_values('deaths_percent',ascending=False)

,population,cases,deaths,deaths_percent,safe_cases
Norte,14752784,89017773,2386633,16.177509,97.318925
Nordeste,48946190,184449738,5812513,11.875312,96.848728
Centro-Oeste,13085364,71590231,1518353,11.603445,97.879106
Sudeste,80421777,224057624,9326839,11.597405,95.837303
Sul,24522343,72207079,1464191,5.970845,97.972233


In [ ]:
"""
Ordenando por casos detectados e curados.
"""
dt.sort_values('safe_cases',ascending=False)

,population,cases,deaths,deaths_percent,safe_cases
Sul,24522343,72207079,1464191,5.970845,97.972233
Centro-Oeste,13085364,71590231,1518353,11.603445,97.879106
Norte,14752784,89017773,2386633,16.177509,97.318925
Nordeste,48946190,184449738,5812513,11.875312,96.848728
Sudeste,80421777,224057624,9326839,11.597405,95.837303


# OPERAÇÃO DE LINHA E AGREGAÇÃO

In [ ]:
import pandas as pd
!curl -o brazil_covid19.tar.gz -s -OL 'https://drive.google.com/uc?export=download&id=1gsOhZkvDKCiEwtjwJbcyyXdbhjyOIukn'
!tar -zxvf brazil_covid19.tar.gz
!rm brazil_covid19.tar.gz

brazil_covid19/
brazil_covid19/brazil_covid19_cities.csv
brazil_covid19/brazil_population_2019.xlsx


In [ ]:
df_cities = pd.read_csv('brazil_covid19/brazil_covid19_cities.csv')
df_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1359080 entries, 0 to 1359079
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   date    1359080 non-null  object 
 1   state   1359080 non-null  object 
 2   name    1359080 non-null  object 
 3   code    1359080 non-null  float64
 4   cases   1359080 non-null  int64  
 5   deaths  1359080 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 62.2+ MB


In [ ]:
df_cities.date = pd.to_datetime(df_cities.date)

In [ ]:
df_cities.date

0         2020-03-27
1         2020-03-27
2         2020-03-27
3         2020-03-27
4         2020-03-27
             ...    
1359075   2020-11-25
1359076   2020-11-25
1359077   2020-11-25
1359078   2020-11-25
1359079   2020-11-25
Name: date, Length: 1359080, dtype: datetime64[ns]

In [ ]:
# Agrupamento por data e estado, mostrando o somatório casos e mortes
df_states = df_cities.groupby(['date','state'])[['cases','deaths']].sum().reset_index()
df_states

,date,state,cases,deaths
0,2020-03-27,AC,0,0
1,2020-03-27,AL,0,0
2,2020-03-27,AM,0,0
3,2020-03-27,AP,0,0
4,2020-03-27,BA,0,0
...,...,...,...,...
6583,2020-11-25,RS,306335,6639
6584,2020-11-25,SC,328894,3561
6585,2020-11-25,SE,88100,2288
6586,2020-11-25,SP,1224544,41601


In [ ]:
# Porporção de mortes por casos
df_states['proporcao'] = df_states.apply(lambda row: 0 if row['cases'] == 0 else 100*row['deaths']/row['cases'], axis=1)

In [ ]:
df_states[df_states['date'].eq(df_states.date.max())].sort_values('proporcao',ascending = False)

,date,state,cases,deaths,proporcao
6579,2020-11-25,RJ,343995,22256,6.469861
6576,2020-11-25,PE,178086,8971,5.037454
6586,2020-11-25,SP,1224544,41601,3.397265
6566,2020-11-25,CE,290887,9530,3.276186
6580,2020-11-25,RN,88064,2665,3.026208
6563,2020-11-25,AM,175298,4826,2.753026
6573,2020-11-25,MT,156128,4055,2.597228
6585,2020-11-25,SE,88100,2288,2.597049
6574,2020-11-25,PA,267700,6880,2.570041
6562,2020-11-25,AL,93907,2318,2.468400
